#### 1. Form a dataframe using the first group of product names: 

In [147]:
from fuzzywuzzy import fuzz
import pandas as pd
from numpy import triu, ones
import numpy as np

t1 = "Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (Up To 95Mb/S Read)Memory Card"
t2 = "Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb Sdsdxpa 064G"
t3 = "Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk Sdsdxxg 064G X46)95 90Mb/S Read Write Speed Video Speed C10 U3 V30"

tg1 = "SANDISK SDC CLASS 10 EXTREME PRO 95MB/S SERIES 64GB"
tg2 = "SANDISK SDC CLASS 10 EXTREME PRO UHS-II 275MB/S SERIES 64GB"

In [148]:
s1 = pd.Series([t1,t2,t3], index=range(3))

In [149]:
dfs = pd.DataFrame([s1.values.tolist()] * s1.shape[0], index=s1.index, columns=s1.index)
dfs = dfs.applymap(lambda x: list([x]))

In [150]:
dfd = dfs + dfs.T
dfd = dfd.mask(triu(ones(dfd.shape)).astype(bool))
dfd

,0,1,2
0,NaN,NaN,NaN
1,[Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (U...,NaN,NaN
2,[Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (U...,[Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb S...,NaN


#### 2. Compute the similarity among product names in the first group using fuzzywuzzy: 

In [151]:
def dist(ss):
    if type(ss) == list:
        return fuzz.partial_ratio(ss[0], ss[1])  

dfg = dfd.applymap(dist)
dfg

,0,1,2
0,NaN,NaN,None
1,54.0,NaN,None
2,56.0,70.0,None


#### 3. Group similar objects (which are assumed to be the same product)

In [152]:
grouplist = []
for i, row in dfd[dfg > 50].iterrows():
    for r in row.values.tolist():
        if type(r) == list:
            grouplist.append(r)

In [153]:
def merge(lsts):
    sets = [set(lst) for lst in lsts if lst]
    merged = 1
    while merged:
        merged = 0
        results = []
        while sets:
            common, rest = sets[0], sets[1:]
            sets = []
            for x in rest:
                if x.isdisjoint(common):
                    sets.append(x)
                else:
                    merged = 1
                    common |= x
            results.append(common)
        sets = results
    return sets

In [154]:
grouplist = merge(grouplist)

#### 4. Form the dataframe with similar above to compute the similarity between all the product names in the first and second group 

In [155]:
s1 = pd.Series([t1,t2,t3], index=range(3))
s2 = pd.Series([tg1,tg2], index=range(2))

In [156]:
dfs1 = pd.DataFrame([s1.values.tolist()] * s2.shape[0], index=range(s2.shape[0]), columns=s1.index)
dfs1 = dfs1.applymap(lambda x: list([x]))
dfs1

,0,1,2
0,[Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (U...,[Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb S...,[Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk ...
1,[Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (U...,[Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb S...,[Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk ...


In [157]:

dfs2 = pd.DataFrame([s2.values.tolist()] * s1.shape[0], index=range(s1.shape[0]), columns=s2.index)
dfs2 = dfs2.applymap(lambda x: list([x]))
dfs2.T

,0,1,2
0,[SANDISK SDC CLASS 10 EXTREME PRO 95MB/S SERIE...,[SANDISK SDC CLASS 10 EXTREME PRO 95MB/S SERIE...,[SANDISK SDC CLASS 10 EXTREME PRO 95MB/S SERIE...
1,[SANDISK SDC CLASS 10 EXTREME PRO UHS-II 275MB...,[SANDISK SDC CLASS 10 EXTREME PRO UHS-II 275MB...,[SANDISK SDC CLASS 10 EXTREME PRO UHS-II 275MB...


In [158]:
dfd2 = dfs1 + dfs2.T

#### 5.  compute the similarity between all the product names in the first and second group

In [159]:
dfg2 = dfd2.applymap(dist)
dfg2

,0,1,2
0,32,24,20
1,15,23,17


#### 6. Find the strongest similarity between the elements in the grouplist and the elements in the second group and match the product names 

In [170]:
vert = dfd2[dfg2 == dfg2.max()]

In [171]:
vert

,0,1,2
0,[Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (U...,[Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb S...,[Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk ...
1,NaN,NaN,NaN


In [175]:
vert.iloc[0,:][vert.iloc[0,:].notnull()]

0    [Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (U...
1    [Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb S...
2    [Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk ...
Name: 0, dtype: object

In [179]:
candidates = {}
for i, row in vert.iterrows():
    for r in row[row.notnull()]:
        candidates.setdefault(r[1], []).append(r[0])
candidates

{'SANDISK SDC CLASS 10 EXTREME PRO 95MB/S SERIES 64GB': ['Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (Up To 95Mb/S Read)Memory Card',
  'Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb Sdsdxpa 064G',
  'Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk Sdsdxxg 064G X46)95 90Mb/S Read Write Speed Video Speed C10 U3 V30']}

In [182]:
grouplist[0]

{'Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (Up To 95Mb/S Read)Memory Card',
 'Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk Sdsdxxg 064G X46)95 90Mb/S Read Write Speed Video Speed C10 U3 V30',
 'Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb Sdsdxpa 064G'}

In [187]:
import operator

matched = {}
for key, val in candidates.items():
    d = {}
    for i, group in enumerate(grouplist):
        d.update({i: len(group.intersection(val))})
    g = sorted(d.items(), key=operator.itemgetter(1), reverse=True)[0][0]
    matched.update({key:grouplist[g]})
    
matched

{'SANDISK SDC CLASS 10 EXTREME PRO 95MB/S SERIES 64GB': {'Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (Up To 95Mb/S Read)Memory Card',
  'Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk Sdsdxxg 064G X46)95 90Mb/S Read Write Speed Video Speed C10 U3 V30',
  'Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb Sdsdxpa 064G'}}